In [1]:
import scipy.sparse as sp
import numpy as np
import math
import sys
import matplotlib.pyplot as plt
import socket
from scipy.io import loadmat
from scipy.sparse import diags
from scipy.sparse.linalg import bicgstab
from scipy.sparse import csr_matrix
import scipy.sparse.linalg as  it
from scipy.sparse import random
import os

Iterative Methods Codes from Algorithms

In [2]:
def conjugate_gradient(A, u, b, p,r,rs_old, threshold):
    x = u.copy() # Initial guess x0
    #r = b - A.dot(x)  # Compute r0
    #p = r.copy()  # Set p0 to r0
    #rs_old = np.dot(r, r)
    itr=0;
    c=0
    while(1):
        #print("iteration: ", itr)
        itr+=1
        Ap = A.dot(p)
        alpha = rs_old / p.dot(Ap)
        x += alpha * p
        
        r -= alpha * Ap
        rs_new = np.dot(r, r)
        error= np.linalg.norm(r)
        if error < threshold:
            print("Converged Error in ", itr-1 ," itr: ",error)
            c=1
            break  # Convergence achieved
        if error >10e10:
            print("Diverged")
            break
        beta = rs_new / rs_old
        p = r + beta * p
        rs_old = rs_new
        if itr%9==0: 
            print("Break Error CG",error)
            break
    return x,p,r,rs_old,c

def bicgstab_(A,u,b,p,r,rs_old,r_star,threshold):
    #print("Bi-Conjugate Gradient Stablized Algorithm")
    x = u.copy() # Initial guess x0
    #r = b - A.dot(x)  # Compute r0
    #p = r.copy()  # Set p0 to r0
    #r_star=r.copy()
    #rs_old = np.dot(r, r_star)
    itr=0;
    c=0
    while(1):
#         print("")
#         print("r[0]: ",r[0])
        Ap=A.dot(p)
        alpha=rs_old/Ap.dot(r_star)
#         print("Printing p[0]: ",p[0]);
#         print("alpha: rs_old/Ap.dot(r_star)",rs_old, "/",Ap.dot(r_star), " = ",alpha)
        s=r-alpha*Ap
#         print("Printing s: ")
#         print("r[0] - alpha*Ap[0]] = ",r[0],"-",alpha,"*",Ap[0])
        As=A.dot(s)
        w=As.dot(s)/As.dot(As)
        x= x+ alpha*p 
        x= x+ w*s
        w_As=w*As
        r = s - w_As
        rs_new = np.dot(r,r_star)
        error= np.linalg.norm(r)
#         print("s[0] - w_As[0]", s[0],"-",w_As[0]);
        if error < threshold:
            print("Convergence Error in ", itr ," itr: ",error)
            c=1
            break  # Convergence achieved
        beta = (rs_new/rs_old)*(alpha/w)
        p=  r + beta*(p-w*Ap)
#         print("r + beta*(p-w*Ap) = ",r[0],"+",beta,"*(",p[0],"-",w,"*",Ap[0])
        rs_old=rs_new
        print("Error in ",itr, " itr: ",error)
        itr+=1
        if (itr==10): 
#             print("Break Error BiCG",error)
#             break
            ;
        
    return x,p,r,rs_old,c 

def jacobiit(A,u,b,threshold):
    print("Jacobi Iteration Algorithm")
    dia_A, rnd_A = separate_diagonal(A)
    D_inv = np.linalg.inv(dia_A)
    T = D_inv.dot(rnd_A)
    x=u.copy()
    itr=0
    while(1):
        itr+=1
        x_new = np.dot(D_inv,b) - np.dot(T,x)
        error= np.linalg.norm(x_new-x)
        x=x_new
        if error < threshold:
            print("Error in ", itr ," itr: ",error)
            break  # Convergence achieved
        if error > 10e10:
            print("Diverged")
            break
        print("Error in ",itr, " itr: ",error)
    return x

The following functions are to export SuiteSparse matrices in .txt files.
Returns values, colidx, offsets,param (nnz,rows), and input txt. These files will be used in the C++ code to read data from. 

In [3]:
def writeFile(s,arr_name,arr,size):
    file=open(s,'w')
    #file.write(arr_name+"={")
    for i in range(size):
        if(i == size-1):
            q=str(arr[i])#+'}'
            file.write (q)
        else: 
            q=str(arr[i])+' '
            file.write(q)
    file.close()

In [4]:
def ipFile(offsets,colIdx,values,param,ipfile):
    #print('asd nk')
    file =open(ipfile,'w')
    for i in range(6):
        if (i==0): 
            file.write(offsets+'\n')
        if (i==1): 
            file.write(colIdx+'\n')
        if (i==2): 
            file.write(values+'\n')
        if (i==3): 
            file.write("vec.txt"+'\n')
        if (i==4): 
            file.write("latency.txt"+'\n')
        if (i==5): 
            file.write(param+'\n')
    file.close()

In [5]:
# Define the matrix size
def MatrixGenerateSS(sparse_matrix,offsets_,colIdx_,values_,param_,ipfile):
    
    rows = sparse_matrix.shape[0]
    cols = rows
    nnz = sparse_matrix.nnz
    density = nnz/(rows*cols)
    offsets =np.empty(rows, dtype=int)
    nonzero_per_row = sparse_matrix.getnnz(axis=1)
    row_indices = sparse_matrix.nonzero()[0]
    col_indices = sparse_matrix.nonzero()[1]
    values =np.empty(nnz, dtype=float)
    for i in range(nnz):
        values[i] = sparse_matrix.data[i]
        
    for j in range(rows):
        if(j==0):
            offsets[0]=nonzero_per_row[0]
            continue
        offsets[j]=nonzero_per_row[j]+offsets[j-1]
        
    param =np.empty(2,dtype=int)
    param[1]=sparse_matrix.shape[0]
    param[0]=sparse_matrix.nnz
    
    writeFile(offsets_, 'offsets', offsets, rows)
    writeFile(colIdx_, 'colIdx', col_indices, nnz)
    writeFile(values_, 'values', values, nnz)
    writeFile(param_, 'param', param, 2)
    ipFile(offsets_,colIdx_,values_,param_,ipfile)

In [6]:
def getXnumberOfFilesSS(chunk,matrix):
    
    for i in range(1):
        offsets_='offsets'+matrix+'.txt'
        colIdx_='colIdx'+matrix+'.txt'
        values_='values'+matrix+'.txt'
        ipfile='ipfiles'+matrix+'.txt'
        param_='param'+matrix+'.txt'
        MatrixGenerateSS(chunk,offsets_,colIdx_,values_,param_,ipfile)

The following functions are to export toy matrices. 

In [7]:
def ndarray_to_csr(matrix):
    # Initialize lists to store values, column indices, and offsets
    values = []
    col_indices = []
    offsets = []  # The first offset is always 0

    # Iterate over each row of the matrix
    for i in range(matrix.shape[0]):
        # Find non-zero elements in the row
        non_zero_indices = np.nonzero(matrix[i])[0]

        # Append non-zero values and their column indices to the respective lists
        values.extend(matrix[i, non_zero_indices])
        col_indices.extend(non_zero_indices)

        # Update the offset for the next row
        offsets.append(len(values))

    # Convert lists to numpy arrays
    values = np.array(values)
    col_indices = np.array(col_indices)
    offsets = np.array(offsets)

    return values, col_indices, offsets

# Sample NumPy array representing a matrix

def save_csr_to_txt(values, col_indices, offsets,s, directory="."):
    # Save values to a text file with space delimiter
    with open(f"{directory}/values"+s+".txt", "w") as f:
        f.write(' '.join(map(str, values)
        ))

    # Save column indices to a text file with space delimiter
    with open(f"{directory}/ind"+s+".txt", "w") as f:
        f.write(' '.join(map(str, col_indices)))

    # Save offsets to a text file with space delimiter
    with open(f"{directory}/offsets"+s+".txt", "w") as f:
        f.write(' '.join(map(str, offsets)))

def chunkCSR(T, chunk_size):
    # Ensure chunk size is valid
    assert T.shape[0] >= chunk_size and T.shape[1] >= chunk_size, "Chunk size exceeds matrix dimensions"

    # Extract chunk of the matrix
    chunk_data = T[:chunk_size, :chunk_size]

    # Convert chunk to CSR format
    chunk_csr = csr_matrix(chunk_data)

    return chunk_csr

def diagDom(matrix):
    # Check if the matrix is CSR format
    assert isinstance(matrix, csr_matrix), "Input matrix must be in CSR format"

    # Get the diagonal elements
    diagonal = matrix.diagonal()

    # Check if all off-diagonal elements are zero
    off_diagonal_sum = matrix.sum() - diagonal.sum()

    return off_diagonal_sum == 0

In [8]:
# ##Running BiCG-STAB on matrix A



In [13]:
file_path = "A_4096.txt"
matrix = np.loadtxt(file_path)
values, col_indices, offsets = ndarray_to_csr(matrix)
print("Values:", values)
print("Column Indices:", col_indices)
print("Offsets:", offsets)
save_csr_to_txt(values, col_indices, offsets,"A4096")

[[0.         0.         0.         0.         0.46224269]
 [0.         0.42134066 0.16578191 0.         0.        ]
 [0.7828573  0.         0.         0.         0.        ]
 [0.         0.98727827 0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]]


FileNotFoundError: A_4096.txt not found.

SuiteSparse Matrices

In [20]:
chunk=4096
b = np.ones(chunk)
u0 = np.ones(chunk)
matrices = []
#matrices= ['2cubes_sphere_ex', 'light_in_tissue_ex','offshore_ex','TEM152078_ex','poisson3Db_ex',
#           'windtunnel_evap3d_ex', 'atmosmodm_ex', 'ifiss_mat_ex' , 'mono_500Hz_ex' ]  ##Add more matrices in the array. The code will output the csr format in .txt
# matrices=['finan512_ex','ASIC_100k_ex','crystm03_ex','epb3_ex','qa8fm_ex','scircuit_ex',
#           'thermomech_TC_ex','xenon1_ex']
# matrices = ['G2_circuit_ex', 'GaAsH6_ex','Pres_Poisson_ex','Si34H36_ex','Trefethen_20000_ex',
#             'bcircuit_ex','ns3Da_ex']
# matrices = ['torso2_ex','wang3_ex']


ps="4k"
for matrix in matrices:
    matx = loadmat(matrix)
    S = matx['matx']
    T = S.tocoo().tocsr()
    T.data = T.data.astype(np.float32)
    U=chunkCSR(T,chunk)
    print("NNZ in ",chunk ,"portion of", matrix, "=",U.nnz)
    print("DIM in ",chunk ,"portion of", matrix,"=",U.shape[0])
    getXnumberOfFilesSS(U, matrix+ps); 
    #u, info = it.cg(U, b, x0=u0)
    #print(u)
    #print(matrix," Diverged for cg? ",info)
    #u, info = it.bicgstab(U, b, x0=u0)
    #print(u)
    #print(matrix," Diverged for BiCG? ",info)
    #dia = diagDom(U)
    #print(matrix," Strictly Diagonally Dominant ? ",int(dia))
    print("().txt exported for '",matrix,chunk,"' dataset");print("");

In [38]:

cfg_name = "cfg1.txt"
matrix = "ASIC_100K"
nnz = 104274
filepath = 'D:\\University\\UMD\Ph.D\\ItSolvers\\C_Sim'
command = "cd " + filepath
print(command)
os.system(command)

command = "main " + cfg_name + " " + matrix + " "+ str(nnz)
output = os.popen(command).read()
print(command)
# Print the output
print(output)

cd D:\University\UMD\Ph.D\ItSolvers\C_Sim
main cfg1.txt ASIC_100K 104274



## The following cells are specific for review_Acamar and calculates the area, power and load imbalance information

In [67]:
chunk=4096
b = np.ones(chunk)
u0 = np.ones(chunk)
matrices = []
#matrices= ['2cubes_sphere_ex', 'light_in_tissue_ex','offshore_ex','TEM152078_ex','poisson3Db_ex',
#           'windtunnel_evap3d_ex', 'atmosmodm_ex', 'ifiss_mat_ex' , 'mono_500Hz_ex' ]  ##Add more matrices in the array. The code will output the csr format in .txt
# matrices=['finan512_ex','ASIC_100k_ex','crystm03_ex','epb3_ex','qa8fm_ex','scircuit_ex',
#           'thermomech_TC_ex','xenon1_ex']
# matrices = ['G2_circuit_ex', 'GaAsH6_ex','Pres_Poisson_ex','Si34H36_ex','Trefethen_20000_ex',
#             'bcircuit_ex','ns3Da_ex']
#matrices = ['torso2_ex','wang3_ex']
matrices= ['TEM152078_ex']

ps="4k"
for matrix in matrices:
    matx = loadmat(matrix)
    S = matx['matx']
    T = S.tocoo().tocsr()
    T.data = T.data.astype(np.float32)
    U=chunkCSR(T,chunk)
    print("NNZ in ",chunk ,"portion of", matrix, "=",U.nnz)
    print("DIM in ",chunk ,"portion of", matrix,"=",U.shape[0])
    print("nnz 1st row: ", U[0].getnnz(axis=1))
    max_length=list()
    min_length=list()
    
    for i in range(32):
        maxx=0
        minn=0
        for j in range(128):
            nnz = U[j].getnnz(axis=1)
            #if (i==30):
             #   print("in 30th: ", nnz[0])
            if (nnz[0] > maxx):
                maxx=nnz
            else:
                minn=nnz
        max_length.append(maxx[0])
        min_length.append(minn[0])
        
    print(max_length)
    print(min_length)
    print("().txt exported for '",matrix,chunk,"' dataset");print("");

NNZ in  4096 portion of TEM152078_ex = 133192
DIM in  4096 portion of TEM152078_ex = 4096
nnz 1st row:  [32]
[74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74]
[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]
().txt exported for ' TEM152078_ex 4096 ' dataset




# SCRAP CODE---FOR TESTING

In [38]:
A =U #.tocsc()

num_rows = A.shape[0]
print(num_rows)

b = np.ones(num_rows)
u0 = np.ones(num_rows)
# Solve the linear system using CG algorithm
u, info = it.cg(A, b, x0=u0)
#print(u)
print("Diverged for cg? ",info)
u, info = it.bicgstab(A, b, x0=u0)
#print(u)
print("Divergedfor BiCG? ",info)



256
Diverged for cg?  0
Divergedfor BiCG?  0


In [ ]:
A=T
u = np.ones(T.shape[0],dtype=np.float32)
b = np.ones(T.shape[0],dtype=np.float32)
r = b - A.dot(u)
p = r.copy()
rs_old = np.dot(r, r)
r_star=r.copy()
conv=0
solution_,p,r,rs_old,conv=bicgstab_(A,u,b,p,r,rs_old,r_star,0.0001)


#solution_,p,r,rs_old,conv= conjugate_gradient(A, u, b, p,r,rs_old,0.0001)
print (solution_)

#solution_,p,r,rs_old = conjugate_gradient(A, u, b, p,r,rs_old,0.0001)
#print(solution_)
jj=0
# while (jj!=1000):
#     if (jj<20):
#         solution_,p,r,rs_old,conv=bicgstab_(A,solution_,b,p,r,rs_old,r_star,0.0001)
#         #solution_,p,r,rs_old,conv = conjugate_gradient(A, solution_, b, p,r,rs_old,0.0001)
#         if (jj==19):
#             u = np.ones(T.shape[0])
#             b = np.ones(T.shape[0])
#             r = b - A.dot(u)
#             p = r.copy() 
#             rs_old = np.dot(r, r)
#             r_star=r.copy()
#         if(conv ==1):break
            
    
#     else:
#         solution_,p,r,rs_old,conv= conjugate_gradient(A, solution_, b, p,r,rs_old,0.0001)

#         #solution_,p,r,rs_old,conv=bicgstab_(A,solution_,b,p,r,rs_old,r_star,0.0001)
#         if(conv ==1): break
#     jj+=1
#     if (conv==1):
#         break
    
print("total iterations: ", jj*10)

In [70]:
152.97+13.57+11.49+5+3.59+20.16+1.39

208.17

In [71]:
208.17+5.27+2.99+15+32+31.67+21.41+13.90

330.41

In [74]:
 340+ 40.81+29.66+17.53+19.88+12.60

460.48

In [75]:
179+71.83

250.82999999999998

In [76]:
25.50+27.95+24.81+30.56+17.53+19.88

146.23000000000002

In [77]:
pow(5.11653e+44,1/32)

24.954879961403282

In [78]:
2*51148/34062

3.003229405202278

In [80]:
3*100/25

12.0

In [81]:
2 flops = 1cc and 1 cc = 66Mhz

SyntaxError: invalid syntax (<ipython-input-81-8f03fd0060ab>, line 1)

In [82]:
2/66

0.030303030303030304

In [ ]:
0.030 FLOPS/s